In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [5]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - There is no [win32] chromedriver for browser 86.0.4240 in cache
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/86.0.4240.22/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\hp\.wdm\drivers\chromedriver\win32\86.0.4240.22]


In [7]:
# Extract description of a post from Instagram link
driver.get('https://www.instagram.com/p/B4SY4cNgt8R/')
soup = BeautifulSoup(driver.page_source,"lxml")
desc = " "
 
for item in soup.findAll('a'):
    desc= desc + " " + str(item.string)
 
# Extract tag list from Instagram post description
taglist = desc.split()
taglist = [x for x in taglist if x.startswith('#')]
index = 0
while index < len(taglist):
    taglist[index] = taglist[index].strip('#')
    index += 1
 
print(taglist)

['dividendstocks', 'dividends', 'dividendsm', 'indianstocks', 'indianstockmarket', 'investor', 'stocksignals', 'stockstrader', 'entrepreneurlife', 'nifty', 'niftyfifty', 'zerodha', 'investment', 'dalalstreet', 'buissnessmindset', 'thelocaltrader']


In [8]:
# Define dataframe to store hashtag information
tag_df  = pd.DataFrame(columns = ['Hashtag', 'Number of Posts','Posting Freq (mins)'])
 
# Loop over each hashtag to extract information
for tag in taglist:
     
    driver.get('https://www.instagram.com/explore/tags/'+str(tag))
    soup = BeautifulSoup(driver.page_source,"lxml")
     
    # Extract current hashtag name
    tagname = tag
    # Extract total number of posts in this hashtag
    # NOTE: Class name may change in the website code
    # Get the latest class name by inspecting web code
    try:
        nposts = soup.find('span', {'class': 'g47SY'}).get_text()
    except AttributeError:
        print("ok")
         
    # Extract all post links from 'explore tags' page
    # Needed to extract post frequency of recent posts
    myli = []
    for a in soup.find_all('a', href=True):
        myli.append(a['href'])
 
    # Keep link of only 1st and 9th most recent post 
    newmyli = [x for x in myli if x.startswith('/p/')]
    del newmyli[:9]
    del newmyli[9:]
    del newmyli[1:8]
    
    timediff = []
 
    # Extract the posting time of 1st and 9th most recent post for a tag
    for j in range(len(newmyli)):
        driver.get('https://www.instagram.com'+str(newmyli[j]))
        soup = BeautifulSoup(driver.page_source,"lxml")
 
        for i in soup.findAll('time'):
            if i.has_attr('datetime'):
                timediff.append(i['datetime'])
                #print(timediff)
                
        # Calculate time difference between posts
        # For obtaining posting frequency
        datetimeFormat = '%Y-%m-%dT%H:%M:%S.%fZ'
        date1 = timediff[1]
        date2 = timediff[0]
        diff = datetime.datetime.strptime(date1, datetimeFormat)\
        - datetime.datetime.strptime(date2, datetimeFormat)
        pfreq= int(diff.total_seconds()/(9*60))

     
    # Add hashtag info to dataframe
    tag_df.loc[len(tag_df)] = [tagname, nposts,pfreq]
    
# Check the final dataframe
print(tag_df)
 
# CSV output for hashtag analysis
tag_df.to_csv('hashtag_list.csv')

ok
              Hashtag Number of Posts Posting Freq (mins)
0      dividendstocks          78,453                   0
1           dividends         132,557                   0
2          dividendsm         132,557                   0
3        indianstocks          21,663                   3
4   indianstockmarket         123,069                   0
5            investor       6,769,178                   0
6        stocksignals          24,802                   0
7        stockstrader          66,990                   0
8    entrepreneurlife      15,123,823                   0
9               nifty         346,452                   0
10         niftyfifty         444,653                   0
11            zerodha          46,631                   0
12         investment      13,806,575                   0
13        dalalstreet          84,793                   3
14   buissnessmindset           1,976                   0
15     thelocaltrader             300                   3
